In [1]:
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import random
random.seed(0)

In [2]:
from itertools import combinations

# generate random combination with number n
def combine(temp_list, n):
    return list(combinations(temp_list, n))

In [3]:
# generate random sublist from current list
def generate_new_list(original_list):
    new_list = []
    for element in original_list:
        if random.choice([True, False]):
            new_list.append(element)
    return new_list

In [4]:
# set semantics and compute strength of arguments

filename = "../../bags/td_qbaf.bag"
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG(filename)
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 12.199999999999973
Argument(name=s0, weight=0.5, strength=0.664592759656642)
Argument(name=s1, weight=0.5, strength=0.664592759656642)
Argument(name=s2, weight=0.5, strength=0.664592759656642)
Argument(name=s3, weight=0.5, strength=0.8135424356888933)
Argument(name=s4, weight=0.5, strength=0.7041994642766211)
Argument(name=s5, weight=0.5, strength=0.7041994642766211)
Argument(name=s6, weight=0.5, strength=0.5000000446420562)
Argument(name=s7, weight=0.5, strength=0.8254237055377077)
Argument(name=s8, weight=0.5, strength=0.7235959804104234)
Argument(name=s9, weight=0.5, strength=0.7235959804104234)
Argument(name=s10, weight=0.5, strength=0.7235959804104234)
Argument(name=c0, weight=0.0, strength=0.7000991119110755)
Argument(name=c1, weight=0.0, strength=0.46753135307445565)
Argument(name=c2, weight=0.0, strength=0.8313656875032375)
Argument(name=c3, weight=0.0, strength=4.617127656482882e-07)
Argument(name=c4, weight=0.0, strength=3.570538632983414e-07)


0.0009878603392623568

In [5]:
# get all the attacks and supports in QBAF
att_relation = model.BAG.get_attacks()
sup_relation = model.BAG.get_supports()
all_relation = att_relation + sup_relation

In [6]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))

In [7]:
topic_a = "c5"
topic_arg = model.BAG.arguments[topic_a]

In [8]:
# compute strength for all arguments
for arg in model.BAG.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('s0', 0.5, 0.664592759656642)
('s1', 0.5, 0.664592759656642)
('s2', 0.5, 0.664592759656642)
('s3', 0.5, 0.8135424356888933)
('s4', 0.5, 0.7041994642766211)
('s5', 0.5, 0.7041994642766211)
('s6', 0.5, 0.5000000446420562)
('s7', 0.5, 0.8254237055377077)
('s8', 0.5, 0.7235959804104234)
('s9', 0.5, 0.7235959804104234)
('s10', 0.5, 0.7235959804104234)
('c0', 0.0, 0.7000991119110755)
('c1', 0.0, 0.46753135307445565)
('c2', 0.0, 0.8313656875032375)
('c3', 0.0, 4.617127656482882e-07)
('c4', 0.0, 3.570538632983414e-07)
('c5', 0.0, 0.8996854332891212)


In [9]:
# approximate attributions
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations
sample_count_N = 1000 # iterate 1000 times

for i in range(len(all_relation_new)):
    print(f"edge i:{i}")

    # split current and the other relation
    cur_relation = all_relation_new[i]
    rest_relation = all_relation_new.copy()
    rest_relation.remove(cur_relation)

    difference = []
    coefficient = []

    sample_count = sample_count_N

    for index in range(1, sample_count+1): # iterations
        new_list = generate_new_list(rest_relation) # choose random elements in rest_relation
        if new_list == []:
            new_tuple = ()
        else:
            new_tuple = tuple(new_list)
        j = new_tuple
        for k in j:
            # remove att
            if k[0] == "att":
                temp_attacker = model.BAG.remove_attack(k[1],k[2]) # k[1] attacks k[2]
            # remove sup
            elif k[0] == "sup":
                temp_supporter = model.BAG.remove_support(k[1],k[2]) # k[1] supports k[2]



        # compute sigma(S U {i})
        model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)
        topic_arg = model.BAG.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength

        # remove cur_relation
        if cur_relation[0]=="att":
            temp_attacker = model.BAG.remove_attack(cur_relation[1],cur_relation[2])
        elif cur_relation[0]=="sup":
            temp_supporter = model.BAG.remove_support(cur_relation[1],cur_relation[2])

        # compute sigma(S)
        model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)
        topic_arg = model.BAG.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)

        # recover BAG for the next iteration
        model = grad.semantics.QuadraticEnergyModel()
        model.approximator = grad.algorithms.RK4(model)
        model.BAG = grad.BAG(filename)
        model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)

    # compute the attribution
    attribution[cur_relation] = sum(difference) / sample_count_N

edge i:0
edge i:1
edge i:2
edge i:3
edge i:4
edge i:5
edge i:6
edge i:7
edge i:8
edge i:9
edge i:10
edge i:11
edge i:12
edge i:13
edge i:14
edge i:15
edge i:16
edge i:17
edge i:18
edge i:19
edge i:20
edge i:21
edge i:22
edge i:23
edge i:24
edge i:25
edge i:26
edge i:27
edge i:28
edge i:29
edge i:30
edge i:31
edge i:32
edge i:33


In [10]:
attribution

{('att', 'c0', 'c1'): -0.0016597054288270765,
 ('att', 'c1', 'c0'): 0.000834336560601931,
 ('att', 'c2', 'c3'): 0.0009286709137880932,
 ('att', 'c3', 'c2'): 0.00016453425954027012,
 ('att', 'c4', 'c5'): -0.02838372220303165,
 ('att', 'c5', 'c4'): 0.015153831234528093,
 ('sup', 's0', 'c0'): -1.7152399525694786e-05,
 ('sup', 'c0', 's0'): 0.0005582498096317925,
 ('sup', 's1', 'c0'): -3.675301631135433e-05,
 ('sup', 'c0', 's1'): 0.0005586867094440162,
 ('sup', 's2', 'c0'): 0.00014488437417198923,
 ('sup', 'c0', 's2'): 0.0005868267695382578,
 ('sup', 's3', 'c1'): 0.0064605936258123504,
 ('sup', 'c1', 's3'): 0.0008818149059085103,
 ('sup', 's7', 'c1'): 0.005974418657512054,
 ('sup', 'c1', 's7'): 0.009541656868788814,
 ('sup', 's3', 'c2'): 0.0012824255533291642,
 ('sup', 'c2', 's3'): 0.0011409803695223682,
 ('sup', 's4', 'c2'): 0.0012060742803774675,
 ('sup', 'c2', 's4'): 0.0006318694819917303,
 ('sup', 's5', 'c2'): 0.0010834220655097282,
 ('sup', 'c2', 's5'): 0.0006701022598286938,
 ('sup', 